In [1]:
"""
Ενοποιώ τα τρία παραγώμενα dataframes
και προσθέτω την στήλη Death_Flag όπου
ορίζω μέσα της την θνησιμότητα. Με την
τιμή μηδέν ορίζεται η επιβίωση και με
την τιμή ένα η θνησιμότητα.

Προσθέτω και την στήλη Specific_Mortality
όπου μέσα σε αυτή δηλώνω μοναδικά για την
κάθε εισαγωγή ασθενή αν επιβίωσε (τιμή 0)
ή όχι (τιμή 1)

9.234 -> Επιβίωσαν στις μονάδες περίθαλψης.
  870 -> Απεβίωσαν στις μονάδες περίθαλψης.

Στοιχεία συνολικού πίνακα
127.803 γραμμές 1.109 στήλες.
"""

'\nΕνοποιώ τα τρία παραγώμενα dataframes\nκαι προσθέτω την στήλη Death_Flag όπου\nορίζω μέσα της την θνησιμότητα. Με την\nτιμή μηδέν ορίζεται η επιβίωση και με\nτην τιμή ένα η θνησιμότητα.\n\nΠροσθέτω και την στήλη Specific_Mortality\nόπου μέσα σε αυτή δηλώνω μοναδικά για την\nκάθε εισαγωγή ασθενή αν επιβίωσε (τιμή 0)\nή όχι (τιμή 1)\n\n9.234 -> Επιβίωσαν στις μονάδες περίθαλψης.\n  870 -> Απεβίωσαν στις μονάδες περίθαλψης.\n\nΣτοιχεία συνολικού πίνακα\n127.803 γραμμές 1.110 στήλες.\n'

In [2]:
import pandas as pd
from numpy import nan

In [3]:
# Εισαγωγή dataframe
df_tables = pd.read_csv('D:\Thesis\exports\o29_HOSP_ICU_merged.csv')
df_ed_patients = pd.read_csv('D:\Thesis\exports\o2_ed_mean_vitals.csv')


df_death= pd.read_csv('D:\Thesis\mimic-iv-1.0\core\Admissions.csv.gz', usecols = ['subject_id', 'hadm_id', 'hospital_expire_flag'])
# Κρατώ μόνο τις εγγραφές που έχει καταγραφεί θάνατος
df_death = df_death.loc[df_death['hospital_expire_flag'] == 1]

In [4]:
# Μετατροπή της στήλης charttime από object σε datetime

df_ed_patients['charttime']= pd.to_datetime(df_ed_patients['charttime'])

#Διαχωρισμός σε δυο καινούργιες στήλες της ημερομηνίας και ώρας
df_ed_patients['date'] = pd.to_datetime(df_ed_patients['charttime']).dt.date
df_ed_patients['time'] = pd.to_datetime(df_ed_patients['charttime']).dt.time

In [5]:
"""
Διαγράφω τις στήλες stay_id και charttime.
Θα παράξουν στήλες που δεν χρειάζομαι.
"""
df_ed_patients = df_ed_patients.drop(['stay_id', 'charttime'], axis=1)

In [6]:
#Συγχωνεύω και ελέγχω αν μετά υπάρχουν διπλές ίδιες στήλες.

df_rslt=pd.merge(df_tables, df_ed_patients, on=['subject_id','hadm_id', 'date', 'time'],
              how="outer", suffixes=('', '_delme'))

# Και αν υπάρχουν τις αφαιρώ.
df_rslt = df_rslt[[c for c in df_rslt.columns if not c.endswith('_delme')]]

In [7]:
# Εισάγω την τιμή ED στα κενά κελιά του Department
df_rslt[['Department']] = df_rslt[['Department']].fillna('ED')

## Δήλωση σημαίας θνησιμότητας στο σύνολο της στήλης

In [8]:
# Ενοποιώ τα dataframes του συνολικού πίνακα με της θνησιμότητας.

df_rslt=pd.merge(df_rslt, df_death, on=['subject_id','hadm_id'],
              how="left", suffixes=('', '_delme'))

In [9]:
#Φέρνω την στήλη θνησιμότητας πρώτη.
cols = list(df_rslt.columns)
cols = [cols[-1]] + cols[:-1]
df_rslt = df_rslt[cols]

# Μετονομασία στήλης θνησιμότητας
df_rslt = df_rslt.rename(columns={"hospital_expire_flag": "Mortality"})

In [10]:
# Εισάγω την τιμή μηδές στα κενά κελιά του Death_Flag
df_rslt[['Mortality']] = df_rslt[['Mortality']].fillna(0)

# Μετατρέπω την στήλη Death_Flag σε ακέραιο.
df_rslt['Mortality'] = df_rslt['Mortality'].apply(int)

## Δήλωση σημαίας θνησιμότητας στην τελευταία εγγραφή κάθε εισαγωγής

In [11]:
"""
Ορισμός νέας στήλης που θα δηλωθεί μόνο στην
τελική εγγραφή για κάθε εισαγωγή ασθενή αν
επιβίωσε ή όχι.
"""
df_rslt.insert(1, "Specific_Mortality", nan)

In [12]:
# Ορισμός της τιμής μηδέν στις εισαγωγές που έχουν επιβιώσει οι ασθενείς
for row in df_rslt.itertuples():
    hadm = row.hadm_id
    temp = df_rslt.loc[df_rslt['hadm_id'] == hadm].sort_values(by=['date', 'time']).tail(1)
    place = temp.index.values
    df_rslt.at[place,'Specific_Mortality'] = 0

In [13]:
for row in df_death.itertuples():
    hadm = row.hadm_id
    temp = df_rslt.loc[df_rslt['hadm_id'] == hadm].sort_values(by=['date', 'time']).tail(1)
    place = temp.index.values
    df_rslt.at[place,'Specific_Mortality'] = 1

In [15]:
#Εξαγωγή σε csv
df_rslt.to_csv('D:\Thesis\exports\o30_ED_HOSP_ICU_merged.csv', index=False)